In [1]:
!pip install --force-reinstall dist/alquimia_fair_forge-0.0.1.tar.gz -q


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from fair_forge.metrics import Context
from fair_forge import Retriever
from fair_forge.schemas import Dataset,Batch
from pydantic import SecretStr

/Users/alexfiorenza/Documents/software_development/projects/alquimia/fair-forge/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
import json

class MyRetriever(Retriever):
    def load_dataset(self) -> list[Dataset]:
        datasets=[]
        with open("dataset.json") as infile:
            for dataset in json.load(infile):
                datasets.append(Dataset.model_validate(dataset)) 
        return datasets

In [18]:
metrics = Context.run(
    MyRetriever,
    judge_api_key=SecretStr("your_api_key")   
)

In [ ]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
mapping_w2v = {
    "mappings": {
        "properties": {
            "word": {"type": "text"},
            "x": {"type": "float"},
            "y": {"type": "float"},
            "z": {"type": "float"},
            "session_id": {"type": "keyword"},
            "assistant_id": {"type": "keyword"}
        }
    }
}

recreate_index(w2v_index, mapping_w2v)

In [ ]:
session_contexts = {}
for item in context.dataset:
    session_id = item.session_id
    context_str = item.context
    language = item.preferred_language
    asssitant_id = item.assistant_id
    session_contexts[session_id]={"context":context_str,"language": language,"assistant_id":asssitant_id}

In [ ]:
mapping_contextualizer = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "context": {"type": "text"},
      "context_insight": {"type": "text"},
      "context_awareness": {"type": "float"},
      "context_thinkings": {"type": "text"},
      "qa_id": {"type": "keyword"},
      "assistant_id": {"type": "keyword"},
    }
  }
}
recreate_index(context_index, mapping_contextualizer)

In [ ]:
docs = []
for metric in metrics:
    docs.append({
            "_index": context_index,
            "_source": metric.model_dump()
    })

helpers.bulk(es, docs)
print(f"Indexed {len(docs)} documents.")